In [1]:
import requests
import os
import time
import numpy as np


def get_sleep_normal(mu: float=1,
                     sigma: float=0.25) -> float:
    return np.random.normal(mu, sigma)

def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # for a period defined by a Gaussian with mu=1 and sigma=0.25.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling
          headers = {'Accept-Language': 'en-US,en;q=0.5'}
          response = self.session.get(url,timeout = self.timeout, headers=headers) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id

In [ ]:

import logging
import sys

project_logger = logging.getLogger()
project_logger.setLevel(logging.INFO)

# Function for ensuring we have all that is needed for logging
def log_message(connector_type, t, dt, url, redirect_url, response_size, response_code, success, error):
    return '{}, {}, {}, {}, {}, {}, {}, {}, {}'.format(connector_type, url, redirect_url, response_size, response_code, success, error)

handler = logging.FileHandler('medicin_prices.log')
handler.setLevel(logging.INFO)
# id;project;connector_type;t;delta_t;url;redirect_url;response_size;response_code;success;error
formatter = logging.Formatter('%(asctime)s, %(message)s')
handler.setFormatter(formatter)
project_logger.addHandler(handler)



In [2]:
# Read in the file products.txt, assumed one product per line
with open('products.txt', 'r') as f:
    products = f.readlines()

# Remove trailing new lines
products = [p.rstrip() for p in products]
print(str(products))

['panodil', 'marevan', 'mianserin', 'syntocinon']


In [3]:
import requests

conn = Connector('medicin_prices.log')

def get_product_info(product_name: str):
    # "http://api.medicinpriser.dk/v1/produkter/virksomtstof/Mianserin?format=json"
    time.sleep(1) # Sleep for 1 second for good measure
    base_url = 'http://api.medicinpriser.dk/v1/produkter/{}?format=json'
    response, _ = conn.get(base_url.format(product_name), 'med-scrape')
    return response


In [ ]:
import os

def has_product_info(vnr):
    return os.

In [4]:
p_info = [get_product_info(pn) for pn in products]

In [5]:
print(str(p_info))
print([p.text for p in p_info])

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
['[{"Navn":"Panodil Zapp","Varenummer":"005775","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005775","Pakning":"10 stk. (blister) filmovertrukne tabl."},{"Navn":"Panodil","Varenummer":"005813","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005813","Pakning":"10 stk. (blister) filmovertrukne tabl."},{"Navn":"Panodil Brus","Varenummer":"005814","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005814","Pakning":"10 stk. brusetabletter"},{"Navn":"Panodil","Varenummer":"008453","Firma":"GlaxoSmithKline Consumer","Styrke":"665 mg","Detaljer":"/v1/produkter/detaljer/008453","Pakning":"100 stk. (dåse) tabl. m modif udløsn"},{"Navn":"Panodil Zapp","Varenummer":"038252","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/038252","Pakning":"100 stk. filmovertrukne tabl

In [6]:
import json

product_json = [json.loads(p.text) for p in p_info]

In [7]:
# start by defining a Project name
project = 'medicin_prices'
import os # generel package for interacting with the system
# among other things automate folder creation
def maybe_create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

maybe_create_dir(project)

subfolders = ['raw_data','parsed_data']

raw_data = os.path.join(project, subfolders[0])
parsed_data = os.path.join(project, subfolders[1])

for directory in subfolders: 
    maybe_create_dir(os.path.join(project, directory))

In [8]:
for p_list in product_json:
    for product in p_list:
        vnr = product['Varenummer']
        file_path = os.path.join(raw_data, vnr + '.json')
        with open(file_path, 'w') as f:
            json.dump(product, f)

In [26]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

import time

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options, executable_path='../../geckodriver')

def get_medication_price_grid_html(vnr: str, sleep_for=10):
    # Create base_url based on vnr
    base_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr=' + vnr
    # Fetch the base_url using the firefox webdriver
    driver.get(base_url)
    # Find the price development link
    price_development = driver.find_element_by_id("ctl00_TopBar_Pridudvikling")
    # Click the price development link
    price_development.click()
    # Find the date_from input box
    date_from_input = driver.find_element_by_id('ctl00_ctl07_ctl00_DateFrom')
    # Click the date_from input field
    date_from_input.click()
    # Clear the date_from input field
    date_from_input.clear()
    # Send the '01-01-1998' key stroke to the date_from input field
    date_from_input.send_keys('01-01-1998')
    # Send key <enter> stroke to the date_from input field
    date_from_input.send_keys(Keys.ENTER)
    # Submit the content of the date_from input field
    date_from_input.submit()
    # Sleep for sleep_for seconds (default=20)
    time.sleep(sleep_for)
    # Copy the page_source into a string var
    return_str = driver.page_source
    # Return the string var to caller
    return return_str

In [10]:
print(product_json)

[[{'Navn': 'Panodil Zapp', 'Varenummer': '005775', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005775', 'Pakning': '10 stk. (blister) filmovertrukne tabl.'}, {'Navn': 'Panodil', 'Varenummer': '005813', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005813', 'Pakning': '10 stk. (blister) filmovertrukne tabl.'}, {'Navn': 'Panodil Brus', 'Varenummer': '005814', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005814', 'Pakning': '10 stk. brusetabletter'}, {'Navn': 'Panodil', 'Varenummer': '008453', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '665 mg', 'Detaljer': '/v1/produkter/detaljer/008453', 'Pakning': '100 stk. (dåse) tabl. m modif udløsn'}, {'Navn': 'Panodil Zapp', 'Varenummer': '038252', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/038252', 'Pakning': '100 stk. filmovertrukne tabl.'}, {'Navn': '

In [11]:
# Create a flattened list of product numbers to scrape
vnrs = [product["Varenummer"] for product_list in product_json for product in product_list]

In [12]:
print('Product numbers: ' + ', '.join(vnrs))

Product numbers: 005775, 005813, 005814, 008453, 038252, 048744, 082830, 083644, 099233, 433126, 550616, 556578, 550624, 550673, 550731, 550830, 557907, 409193, 111955, 409632, 569433, 041225, 041866, 058331, 094311, 082316, 102870, 174939, 144225, 100849, 162353, 395896, 429622, 438219, 136130, 592510, 418815, 183521, 192683, 133949, 572150, 531611, 449211


In [27]:
from tqdm import tqdm_notebook as tqdm # Progressbar module
product_html = []

for i in tqdm(range(0, len(vnrs))):
    product_html.append(get_medication_price_grid_html(vnrs[i]))

In [28]:
import re
from bs4 import BeautifulSoup as bs

def get_price_grid(html_table):
    soup = bs(html_table)
    price_grid_table = soup.findAll('table', attrs={'id': re.compile(r'.*PriceGrid.*')})
    print(len(price_grid_table))
    if (len(price_grid_table) == 1):
        return price_grid_table[0]
    else:
        return None

In [31]:
print(', '.join([str(len(l)) for l in product_html]))


print(len(product_html))
# print(len(price_grid_table))
# price_grid_table[0]

202768, 556125, 202547, 286498, 267420, 775126, 231433, 76634, 204993, 403077, 298488, 204855, 328027, 327156, 326080, 325944, 199511, 326463, 420454, 344692, 344680, 353966, 304183, 315248, 540660, 429538, 536160, 778522, 657771, 657708, 539693, 421437, 658585, 779131, 656712, 539492, 538141, 659138, 460716, 220109, 204842, 342391, 342339
43


In [30]:
tables = [get_price_grid(html_chunk) for html_chunk in product_html]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [33]:
print(len(tables))

43


In [32]:
print(str(tables[14]))

<table border="1" cellspacing="0" class="grid" id="ctl00_ctl07_ctl00_PriceGrid" rules="all" style="border-collapse:collapse;">
<tbody><tr valign="middle">
<th scope="col">Lægemiddel/<br/>prisperiode</th><th scope="col">Tilskudspris (kr.)</th><th scope="col">Panodil Junior 550731 (kr.)</th><th scope="col">Pinex Junior 487959 (kr.)</th>
</tr><tr>
<td>29.12.1997</td><td>-</td><td>4,48</td><td>-</td>
</tr><tr class="odd">
<td>12.01.1998</td><td>-</td><td>4,48</td><td>-</td>
</tr><tr>
<td>26.01.1998</td><td>-</td><td>4,58</td><td>-</td>
</tr><tr class="odd">
<td>09.02.1998</td><td>-</td><td>4,58</td><td>-</td>
</tr><tr>
<td>23.02.1998</td><td>-</td><td>4,58</td><td>-</td>
</tr><tr class="odd">
<td>09.03.1998</td><td>-</td><td>4,58</td><td>-</td>
</tr><tr>
<td>23.03.1998</td><td>-</td><td>4,58</td><td>-</td>
</tr><tr class="odd">
<td>06.04.1998</td><td>-</td><td>4,70</td><td>-</td>
</tr><tr>
<td>20.04.1998</td><td>-</td><td>4,70</td><td>-</td>
</tr><tr class="odd">
<td>04.05.1998</td><td>-</

In [20]:
def make_html(table_html):
    return "<{h}><{b}>{s}</{b}></{h}>".format(h='html', b='body', s=str(table_html))

In [8]:
import pandas as pd

price_grid_df = pd.read_html(make_html(price_grid_table))

In [9]:
price_grid_df[0]

,Lægemiddel/prisperiode,Tilskudspris (kr.),Marevan 569433 (kr.),Marevan 409632 (kr.)
0,29.12.1997,-,-,-
1,12.01.1998,-,-,-
2,26.01.1998,-,-,-
3,09.02.1998,-,-,-
4,23.02.1998,-,-,-
5,09.03.1998,-,-,-
6,23.03.1998,-,-,-
7,06.04.1998,-,-,-
8,20.04.1998,-,-,-
9,04.05.1998,-,-,-
